In [1]:
from universal_agent import invoke
from dotenv import load_dotenv

load_dotenv()

invoke("What is the weather in Tokyo?")

USER_AGENT environment variable not set, consider setting it to identify your requests.
🖇 AgentOps:  WARNING: agentops is out of date. Please update with the command: 'pip install --upgrade agentops'
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=90ece314-7c6c-47e9-abaf-c7655bc857e4


User (to Assistant):

What is the weather in Tokyo?

--------------------------------------------------------------------------------
Assistant (to User):

***** Suggested tool call (call_2x3DFvLpYbuFpGHnAVHrm7Zx): websearch *****
Arguments: 
{"query":"Tokyo weather","max_results":1}
**************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION websearch...
Call ID: call_2x3DFvLpYbuFpGHnAVHrm7Zx
Input arguments: {'query': 'Tokyo weather', 'max_results': 1}
User (to Assistant):

***** Response from calling tool (call_2x3DFvLpYbuFpGHnAVHrm7Zx) *****
Query: Tokyo weather

Sources:

Relevance Score: 0.9716193
URL: https://www.weatherapi.com/
Content: {'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1736923499, 'localtime': '2025-01-15 15:44'}, 'current': {'last_updated_epoc

🖇 AgentOps: Session Stats - Duration: 12.1s | Cost: $0.000402 | LLMs: 3 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=90ece314-7c6c-47e9-abaf-c7655bc857e4


''

In [1]:
from autobuild_superteam import invoke

invoke("What are the latest Sony cameras?")

USER_AGENT environment variable not set, consider setting it to identify your requests.


==> Generating agents...
['Camera_Technology_Expert', 'Photography_Expert', 'Electronics_Expert', 'Digital_Media_Expert', 'Product_Review_Expert'] are generated.
==> Generating system message...
Preparing system message for Camera_Technology_Expert
Preparing system message for Photography_Expert
Preparing system message for Electronics_Expert
Preparing system message for Digital_Media_Expert
Preparing system message for Product_Review_Expert
==> Generating description...
Preparing description for Camera_Technology_Expert
Preparing description for Photography_Expert
Preparing description for Electronics_Expert
Preparing description for Digital_Media_Expert
Preparing description for Product_Review_Expert
==> Creating agents...
Creating agent Camera_Technology_Expert...
Creating agent Photography_Expert...
Creating agent Electronics_Expert...
Creating agent Digital_Media_Expert...
Creating agent Product_Review_Expert...
Adding user console proxy...


🖇 AgentOps:  WARNING: agentops is out of date. Please update with the command: 'pip install --upgrade agentops'
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=f886286f-852c-4734-b008-a254c6295d2e


Computer_terminal (to Task Manager):

What are the latest Sony cameras?

--------------------------------------------------------------------------------

Next speaker: Tools_Assistant

Tools_Assistant (to Task Manager):

***** Suggested tool call (call_EmGaJpxonDSqRNwt1bNGzDzY): websearch *****
Arguments: 
{"query":"latest Sony cameras 2024","max_results":5}
**************************************************************************

--------------------------------------------------------------------------------

Next speaker: Computer_terminal


>>>>>>>> EXECUTING FUNCTION websearch...
Call ID: call_EmGaJpxonDSqRNwt1bNGzDzY
Input arguments: {'query': 'latest Sony cameras 2024', 'max_results': 5}
Computer_terminal (to Task Manager):

***** Response from calling tool (call_EmGaJpxonDSqRNwt1bNGzDzY) *****
Query: latest Sony cameras 2024

Sources:

Relevance Score: 0.9195344
URL: https://cameratimes.org/rumored-sony-camera-roadmap/
Content: Sony is expected to announce several new Sony m

🖇 AgentOps: Session Stats - Duration: 1m 37.3s | Cost: $0.023719 | LLMs: 8 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=f886286f-852c-4734-b008-a254c6295d2e


ChatResult(chat_id=None, chat_history=[{'content': 'What are the latest Sony cameras?', 'role': 'assistant', 'name': 'Computer_terminal'}, {'content': 'None', 'tool_calls': [{'id': 'call_EmGaJpxonDSqRNwt1bNGzDzY', 'function': {'arguments': '{"query":"latest Sony cameras 2024","max_results":5}', 'name': 'websearch'}, 'type': 'function'}], 'name': 'Tools_Assistant', 'role': 'assistant'}, {'content': 'Query: latest Sony cameras 2024\n\nSources:\n\nRelevance Score: 0.9195344\nURL: https://cameratimes.org/rumored-sony-camera-roadmap/\nContent: Sony is expected to announce several new Sony mirrorless cameras, compact vlogging cameras and cinema cameras in 2024. Here is the rumored Sony Camera Roadmap for 2024: Sony a7 V. The Sony a7 V will replce the currently a7 IV and is expected to be released in late 2024 to early 2025. Currently, there are NOT many rumor about the Sony a7 V\n\nRelevance Score: 0.8914433\nURL: https://sonyaddict.com/2024/01/31/coming-for-sony-alpha-cameras-in-2024/\nCont

---
